<a href="https://colab.research.google.com/github/nuravs/supply-chain-predictor/blob/main/supply_chain_predictor_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
"""
Supply Chain Disruption Predictor
=================================

Day 1: Setting up development environment
Day 2: API registrations and first data collector

Author: [Varun]
Date: [4th July 2025]
"""

'\nSupply Chain Disruption Predictor\n=================================\n\nDay 1: Setting up development environment\nDay 2: API registrations and first data collector\n\nAuthor: [Varun]\nDate: [4th July 2025]\n'

In [9]:
# Cell 2: Install Required Libraries (CORRECTED)
!pip install requests beautifulsoup4 pandas schedule feedparser python-dotenv
# Note: sqlite3 is built into Python, no need to install it

In [10]:
# Cell 3: Import Libraries (CORRECTED)
import requests
import pandas as pd
import sqlite3  # This is built into Python
import json
from datetime import datetime, timedelta
import time
import os
import feedparser  # For RSS feeds

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [11]:
# Cell 4: Test Basic Setup (UNCHANGED)
print("✅ All libraries imported successfully!")
print(f"📅 Setup date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🚀 Ready to start building!")

✅ All libraries imported successfully!
📅 Setup date: 2025-07-04 08:54:01
🚀 Ready to start building!


In [17]:
# Cell 5: API Configuration (CORRECTED)
# Store your API keys here (we'll use environment variables later)

# OpenWeatherMap API Key - REPLACE WITH YOUR NEW KEY
OPENWEATHER_API_KEY = "170275dc9a363d20f61b1ef698a0d818"  # Get a new one from OpenWeatherMap

# Test API key function
def test_openweather_api():
    url = f"http://api.openweathermap.org/data/2.5/weather?q=London&appid={OPENWEATHER_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        print("✅ OpenWeatherMap API working!")
        return True
    else:
        print(f"❌ API Error: {response.status_code}")
        return False

# Actually call the test function
print("🔑 Testing API connection...")
api_test_result = test_openweather_api()

🔑 Testing API connection...
✅ OpenWeatherMap API working!


In [18]:
# Cell 5B: Secure API Key Storage (OPTIONAL)
from google.colab import userdata

# Store API key securely in Colab
# Go to the "key" icon in left sidebar → "Secrets" → Add new secret
# Name: OPENWEATHER_API_KEY
# Value: your_actual_api_key

try:
    OPENWEATHER_API_KEY = userdata.get('OPENWEATHER_API_KEY')
    print("✅ API key loaded from secure storage")
except:
    print("⚠️ Using hardcoded API key (not recommended for production)")
    OPENWEATHER_API_KEY = "your_api_key_here"

⚠️ Using hardcoded API key (not recommended for production)


In [13]:
# Cell 6: Weather Data Collector Class
class WeatherCollector:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5"

    def get_weather_by_city(self, city_name):
        """Get current weather for a specific city"""
        url = f"{self.base_url}/weather?q={city_name}&appid={self.api_key}&units=metric"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            # Extract relevant information
            weather_data = {
                'city': data['name'],
                'country': data['sys']['country'],
                'temperature': data['main']['temp'],
                'humidity': data['main']['humidity'],
                'pressure': data['main']['pressure'],
                'wind_speed': data['wind'].get('speed', 0),
                'weather_main': data['weather'][0]['main'],
                'weather_description': data['weather'][0]['description'],
                'timestamp': datetime.now().isoformat()
            }

            return weather_data

        except requests.exceptions.RequestException as e:
            print(f"❌ Error fetching weather data: {e}")
            return None

    def get_weather_multiple_cities(self, cities):
        """Get weather for multiple cities"""
        weather_data = []

        for city in cities:
            print(f"📍 Fetching weather for {city}...")
            data = self.get_weather_by_city(city)
            if data:
                weather_data.append(data)
            time.sleep(1)  # Be respectful to the API (1 second delay)

        return weather_data

# Test the weather collector
print("✅ Weather Collector class created!")

✅ Weather Collector class created!


In [14]:
# Cell 7: Test Weather Collection
# Major port cities for supply chain monitoring
MAJOR_PORTS = [
    "Shanghai",
    "Singapore",
    "Rotterdam",
    "Los Angeles",
    "Hamburg",
    "Dubai"
]

# Initialize weather collector
weather_collector = WeatherCollector(OPENWEATHER_API_KEY)

# Test with one city first
print("🌤️  Testing weather collection...")
test_weather = weather_collector.get_weather_by_city("London")

if test_weather:
    print("✅ Weather collection successful!")
    print(f"📊 Sample data: {test_weather}")
else:
    print("❌ Weather collection failed - check your API key")

🌤️  Testing weather collection...
✅ Weather collection successful!
📊 Sample data: {'city': 'London', 'country': 'GB', 'temperature': 20.37, 'humidity': 48, 'pressure': 1028, 'wind_speed': 2.64, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds', 'timestamp': '2025-07-04T08:54:01.809456'}


In [15]:
# Cell 8: Data Storage Setup
def create_database():
    """Create SQLite database for storing weather data"""
    conn = sqlite3.connect('supply_chain_data.db')
    cursor = conn.cursor()

    # Create weather data table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS weather_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            city TEXT,
            country TEXT,
            temperature REAL,
            humidity INTEGER,
            pressure REAL,
            wind_speed REAL,
            weather_main TEXT,
            weather_description TEXT,
            timestamp TEXT
        )
    ''')

    conn.commit()
    conn.close()
    print("✅ Database created successfully!")

def save_weather_data(weather_data):
    """Save weather data to database"""
    conn = sqlite3.connect('supply_chain_data.db')
    cursor = conn.cursor()

    for data in weather_data:
        cursor.execute('''
            INSERT INTO weather_data
            (city, country, temperature, humidity, pressure, wind_speed,
             weather_main, weather_description, timestamp)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            data['city'], data['country'], data['temperature'],
            data['humidity'], data['pressure'], data['wind_speed'],
            data['weather_main'], data['weather_description'], data['timestamp']
        ))

    conn.commit()
    conn.close()
    print(f"✅ Saved {len(weather_data)} weather records to database")

# Create the database
create_database()

✅ Database created successfully!


In [16]:
# Cell 9: First Data Collection Run
print("🚀 Running first data collection...")

# Collect weather data for major ports
weather_data = weather_collector.get_weather_multiple_cities(MAJOR_PORTS)

if weather_data:
    # Save to database
    save_weather_data(weather_data)

    # Display results
    df = pd.DataFrame(weather_data)
    print("\n📊 Collected Weather Data:")
    print(df[['city', 'country', 'temperature', 'weather_main', 'wind_speed']])

    print(f"\n✅ Successfully collected data for {len(weather_data)} cities!")
else:
    print("❌ No weather data collected - check your API key")

🚀 Running first data collection...
📍 Fetching weather for Shanghai...
📍 Fetching weather for Singapore...
📍 Fetching weather for Rotterdam...
📍 Fetching weather for Los Angeles...
📍 Fetching weather for Hamburg...
📍 Fetching weather for Dubai...
✅ Saved 6 weather records to database

📊 Collected Weather Data:
          city country  temperature weather_main  wind_speed
0     Shanghai      CN        34.58       Clouds        5.04
1    Singapore      SG        31.16       Clouds        3.25
2    Rotterdam      NL        21.65        Clear        4.54
3  Los Angeles      US        18.25        Clear        1.63
4      Hamburg      DE        19.65       Clouds        4.13
5        Dubai      AE        37.51        Clear        5.21

✅ Successfully collected data for 6 cities!
